<a href="https://colab.research.google.com/github/camilla8989/pythonstudy/blob/TextMining/Text_mining_project_work_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get News headlines

In [ ]:
!pip install lxml
!pip install html5lib

import urllib.request
import bs4 as bs
import yfinance as yf
import time
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def get_news(ticker, start_date, end_date):
    current_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
    delta = timedelta(days=1)
    headlines = []

    while current_date <= end_date:
        formatted_date = current_date.strftime('%Y-%m-%d')
        
        # Fetch news from Google News RSS search
        query_date = current_date.strftime('%Y%m%d')
        url = f'https://news.google.com/rss/search?q={ticker}+after%3A{query_date}+before%3A{query_date}&hl=en-US&gl=US&ceid=US:en'
        try:
            doc = urllib.request.urlopen(url).read()
            parsed_doc = bs.BeautifulSoup(doc,'lxml')

            titles = parsed_doc.find_all('title')
            pub_dates = parsed_doc.find_all('pubdate')

            daily_headlines = [{'Headline': title.text, 'Date': formatted_date} for title, pub_date in zip(titles, pub_dates)]
            headlines.extend(daily_headlines)
        except Exception as e:
            print(f"Error on {formatted_date} (Google News): {e}")

        # Fetch news from Yahoo Finance
        url = f"https://finance.yahoo.com/rss/headline?s={ticker}&t={formatted_date}"
        try:
            doc = urllib.request.urlopen(url).read()
            parsed_doc = bs.BeautifulSoup(doc,'xml')

            items = parsed_doc.find_all('item')
            daily_headlines = [{'Headline': item.title.text, 'Date': formatted_date} for item in items]
            headlines.extend(daily_headlines)
        except Exception as e:
            print(f"Error on {formatted_date} (Yahoo Finance): {e}")
        
        current_date += delta
        time.sleep(15)

    return headlines

In [ ]:
ticker = 'AMAZON'
ticker_stock = 'AMZN'
start_date = '2022-08-01'
end_date = '2023-05-01'

headlines = get_news(ticker, start_date, end_date)
headlines.to_csv("output-1.csv", index=False)

# Get Stock data and Label 

In [23]:
import pandas as pd
import yfinance as yf

data = pd.read_csv("output-1.csv")

# Use the "Date" column as an index and combine rows with the same date into one row
data['Date'] = pd.to_datetime(data['Date'])
data = data.groupby('Date').agg(lambda x: ', '.join(set(x.dropna()))).reset_index()

# Get Amazon stock price from yfinance
start_date = data['Date'].min().strftime('%Y-%m-%d')
end_date = data['Date'].max().strftime('%Y-%m-%d')
apple_stock = yf.download('AMZN', start=start_date, end=end_date)

# Calculate daily stock price changes
apple_stock['Price_Change'] = apple_stock['Close'].pct_change()

# Label stock price changes
apple_stock['Label'] = apple_stock['Price_Change'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

# reset index
apple_stock = apple_stock.reset_index()

# Merge datasets
merged_data = pd.merge(data, apple_stock[['Date', 'Label']], on='Date', how='left')

# remove rows without stock prices
merged_data = merged_data.dropna(subset=['Label'])

# convert Label column to integer
merged_data['Label'] = merged_data['Label'].astype(int)


data = merged_data

[*********************100%***********************]  1 of 1 completed


# SENTIMENT SCORE

In [24]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk 
! pip install nltk
nltk.download('vader_lexicon')

# Read in the data
headline = data['Headline']

#calculation of sentiment score
def calc_sentiment_score(headline):
    analyser = SentimentIntensityAnalyzer()
    scores = analyser.polarity_scores(headline)
    return scores['compound']

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [25]:
def calc_sentiment_score_normalized(headline):
    analyser = SentimentIntensityAnalyzer()
    scores = analyser.polarity_scores(headline)
    
    # Define minimum and maximum values for the compound score
    min_score = -1
    max_score = 1
    
    # Normalize the compound score to a range of 0 to 1
    normalized_score = (scores['compound'] - min_score) / (max_score - min_score)
    
    return normalized_score

analyser = SentimentIntensityAnalyzer()

for headline in data['Headline']:
    scores = analyser.polarity_scores(headline)
    

data['sentiment_score'] = data['Headline'].apply(calc_sentiment_score)

In [26]:
#normalized sentiment score

analyser = SentimentIntensityAnalyzer()

for headline in data['Headline']:
    scores = analyser.polarity_scores(headline)
    

data['sentiment_score_normalized'] = data['Headline'].apply(calc_sentiment_score_normalized)
data.dropna(inplace=True)

print(data)

          Date                                           Headline  Label  \
0   2022-08-01  NY21: Castelli says he's the moderate Democrat...      0   
1   2022-08-02  'Road House': Billy Magnussen, Daniela Melchio...     -1   
2   2022-08-03  Roman Kemp Says He 'Contemplated Suicide' In D...      1   
3   2022-08-04  "AMAZON after:20220804 before:20220804" - Goog...      1   
4   2022-08-05  ‘Plucked from thin air’: Paramount plays down ...     -1   
..         ...                                                ...    ...   
266 2023-04-24  Comcast Tiptoes Into E-Commerce. Here's Why It...     -1   
267 2023-04-25  Interview With Value Investor Bill Nygren of O...     -1   
268 2023-04-26  Montauk's Lazybones Makes Fishing So Fluking E...      1   
269 2023-04-27  Uncle Acid and the Deadbeats' Kevin Starrs on ...      1   
270 2023-04-28  ECC's Youngest 2022 Grad Proves Age No Obstacl...     -1   

     sentiment_score  sentiment_score_normalized  
0             0.9625                

# Clean data

In [6]:
#CLEANING THE DATA 
import string
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
nltk.download('stopwords')

# Remove 'Google News' and 'AMAZON after'
data['Headline'] = data['Headline'].str.replace("Google News", "", regex=False)
data['Headline'] = data['Headline'].str.replace("AMAZON after", "", regex=False)

# Remove punctuation marks
merged_data['Headline'] = merged_data['Headline'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

# Convert headlines to lowercase
merged_data['Headline'] = merged_data['Headline'].str.lower()

# Remove stopwords
stop_words = set(stopwords.words('english'))
merged_data['Headline'] = merged_data['Headline'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# Apply stemming
stemmer = PorterStemmer()
merged_data['Headline'] = merged_data['Headline'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

# Remove non-alphabetic characters
merged_data['Headline'] = merged_data['Headline'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))

merged_data.to_csv("output_final.csv", index=False)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Train data

## Logistic Regression

In [10]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Read in the data
data = pd.read_csv('output_final.csv', encoding = "ISO-8859-1")
data.head(1)
data = data[['Headline','Label']]

# Set train set and test set
train, test = train_test_split(data, test_size=0.3)
headlines = train['Headline'].astype(str).tolist()

# Convert to word vector with single word, create sparse matrices 
basicvectorizer = CountVectorizer(ngram_range=(1,1))
basictrain = basicvectorizer.fit_transform(headlines)
print(basictrain.shape)

# Train Model
basicmodel = LogisticRegression()
basicmodel = basicmodel.fit(basictrain, train["Label"])

# Prediction
testheadlines = test['Headline'].astype(str).tolist()
basictest = basicvectorizer.transform(testheadlines)
predictions = basicmodel.predict(basictest)
predictions
pd.crosstab(test["Label"], predictions, rownames=["Actual"], colnames=["Predicted"])

# Evaluation model
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix

print (classification_report(test["Label"], predictions))
print (accuracy_score(test["Label"], predictions))

# Convert to word vector with ngram=2 & ngram=3, create sparse matrices 
basicvectorizer2 = CountVectorizer(ngram_range=(3,3))
basictrain2 = basicvectorizer2.fit_transform(headlines)
print(basictrain2.shape)

basicmodel2 = LogisticRegression()
basicmodel2 = basicmodel2.fit(basictrain2, train["Label"])

basictest2 = basicvectorizer2.transform(testheadlines)
predictions2 = basicmodel2.predict(basictest2)

pd.crosstab(test["Label"], predictions2, rownames=["Actual"], colnames=["Predicted"])

print (classification_report(test["Label"], predictions2))
print (accuracy_score(test["Label"], predictions2))

(131, 9237)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          -1       0.75      0.49      0.59        37
           0       0.00      0.00      0.00         1
           1       0.39      0.68      0.50        19

    accuracy                           0.54        57
   macro avg       0.38      0.39      0.36        57
weighted avg       0.62      0.54      0.55        57

0.543859649122807
(131, 38388)
              precision    recall  f1-score   support

          -1       0.70      0.81      0.75        37
           0       0.00      0.00      0.00         1
           1       0.50      0.37      0.42        19

    accuracy                           0.65        57
   macro avg       0.40      0.39      0.39        57
weighted avg       0.62      0.65      0.63        57

0.6491228070175439


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## BERT

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Convert format
data = pd.read_csv('output_final.csv', encoding = "ISO-8859-1")
data['Headline'] = data['Headline'].astype(str)
data['Label'] = data['Label'] + 1
data['Label'] = data['Label'].astype(int)

# Split the data into training and testing sets with a 7:3 ratio
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

!pip install transformers
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report


class NewsDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        headline = row['Headline']
        label = row['Label']
        inputs = self.tokenizer(headline, return_tensors="pt", max_length=self.max_len, padding=True, truncation=True)
        input_ids = inputs["input_ids"].squeeze()
        attention_mask = inputs["attention_mask"].squeeze()

        return input_ids, attention_mask, label

# Config
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
model.to(device)

max_len = 128
batch_size = 16
epochs = 5

train_dataset = NewsDataset(train_data, tokenizer, max_len)
test_dataset = NewsDataset(test_data, tokenizer, max_len)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-5)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
# Train Model
from typing import List
import torch

def pad_collate_fn(batch: List[torch.Tensor]):
    input_ids, attention_mask, labels = zip(*batch)
    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = torch.tensor(labels, dtype=torch.long)

    return input_ids, attention_mask, labels

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=pad_collate_fn)

for epoch in range(epochs):
    model.train()
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [elem.to(device) for elem in batch]
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

In [13]:
# Evaluation model
from sklearn.metrics import accuracy_score

model.eval()
predictions = []
ground_truth = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [elem.to(device) for elem in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        _, preds = torch.max(logits, 1)
        predictions.extend(preds.cpu().numpy().tolist())
        ground_truth.extend(labels.cpu().numpy().tolist())

# Calculate and print classification report
report = classification_report(ground_truth, predictions)
print("Classification Report:")
print(report)

# Calculate and print accuracy
accuracy = accuracy_score(ground_truth, predictions)
print("Accuracy Score:", accuracy)


Classification Report:
              precision    recall  f1-score   support

           0       0.54      1.00      0.70        30
           2       1.00      0.04      0.07        27

    accuracy                           0.54        57
   macro avg       0.77      0.52      0.38        57
weighted avg       0.76      0.54      0.40        57

Accuracy Score: 0.543859649122807


## SVM

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix

# Preprocess the data as needed
X = data['Headline'].values
y = data['Label'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the data using TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train an SVM classifier with cross-validation
clf = svm.SVC(kernel='linear')
scores = cross_val_score(clf, X_train_tfidf, y_train, cv=5)

# Print the cross-validation scores
print('Cross-validation scores:', scores)
print('Mean accuracy:', scores.mean())

# Train the SVM classifier on the entire training set
clf.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test_tfidf)

# Evaluate the performance of the classifier
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

print("Accuracy:", accuracy_score(y_test, y_pred))



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Cross-validation scores: [0.53333333 0.5        0.53333333 0.46666667 0.5       ]
Mean accuracy: 0.5066666666666666
[[16  5]
 [12  5]]
              precision    recall  f1-score   support

           0       0.57      0.76      0.65        21
           2       0.50      0.29      0.37        17

    accuracy                           0.55        38
   macro avg       0.54      0.53      0.51        38
weighted avg       0.54      0.55      0.53        38

Accuracy: 0.5526315789473685
